- Use the Google word2vec -> extract the words into vector
- Create an ember matrix for training

In [2]:
!pip install Keras-Preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00


#Load data

In [15]:
import pandas as pd
import numpy as np
# Import confusion_matrix and classification_report from the sklearn.metrics module
from sklearn.metrics import confusion_matrix, classification_report

pd.set_option('display.max_colwidth', None)
df = pd.read_csv('/content/Restaurant_Reviews.tsv', sep='\t')
print(df)


                                                                                                                                     Review  \
0                                                                                                                  Wow... Loved this place.   
1                                                                                                                        Crust is not good.   
2                                                                                                 Not tasty and the texture was just nasty.   
3                                                   Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.   
4                                                                               The selection on the menu was great and so were the prices.   
..                                                                                                                                      ...   

#Load model word2vec pretrained

#### Download link



> https://www.kaggle.com/datasets/leadbest/googlenewsvectorsnegative300



In [5]:
!pip install kaggle
!mkdir ~/.kaggle


In [6]:
# Step 1: Upload kaggle.json
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file
# Step 2: Move kaggle.json to the correct location
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Step 3: Set permissions
!chmod 600 ~/.kaggle/kaggle.json
# Install Kaggle package
!pip install kaggle

Saving kaggle.json to kaggle.json


In [7]:
!kaggle datasets download -d leadbest/googlenewsvectorsnegative300

Dataset URL: https://www.kaggle.com/datasets/leadbest/googlenewsvectorsnegative300
License(s): other
100% 3.16G/3.17G [00:44<00:00, 123MB/s]
100% 3.17G/3.17G [00:44<00:00, 76.2MB/s]


In [8]:
!pip install -q unzip

  Preparing metadata (setup.py) ... done


In [9]:
!unzip /content/googlenewsvectorsnegative300.zip

Archive:  /content/googlenewsvectorsnegative300.zip
  inflating: GoogleNews-vectors-negative300.bin  
  inflating: GoogleNews-vectors-negative300.bin.gz  


In [16]:
import pandas as pd
from datetime import date, timedelta
import re
from nltk.tokenize import word_tokenize
from gensim import corpora, models
from gensim.models import KeyedVectors
from gensim.matutils import corpus2dense
import gensim

#Loading the word vectors from Google trained word2Vec model
GoogleModel = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary=True)


This line loads a pre-trained Google Word2Vec model (GoogleNews-vectors-negative300.bin). The Word2Vec model was trained on the Google News dataset, containing 300-dimensional word vectors.

KeyedVectors.load_word2vec_format: Loads the word vectors from a binary file format.
'/content/GoogleNews-vectors-negative300.bin': The file path to the pre-trained model file (in this case, the binary format is specified with binary=True).
binary=True: This specifies that the model is in a binary format, which is more efficient for large models like Google's Word2Vec.


Goal: The code sets up an environment for processing text data, tokenizing it, and loading a pre-trained word embedding model (Google's Word2Vec). The GoogleModel will allow the user to work with 300-dimensional vectors representing word meanings, which can be used for tasks like similarity analysis, text classification, etc.

#Test model

Attempt to detect

In [17]:
word = 'King'
vector = GoogleModel[word]
print(len(vector))

300


Goal: The code retrieves the 300-dimensional word vector for the word "apple" and prints its length.
Key point: Pre-trained Word2Vec models like the Google News model typically represent words as fixed-length vectors, in this case, 300-dimensional. This vector can be used for various NLP tasks, such as finding word similarities, analogies, or clustering.

The dimension of vector will be similar to

In [ ]:
king_vector = np.array([0.2, 0.4, 0.6, 0.3, 0.1, 0.05, 0.9, 0.8, 0.7, 0.4])
queen_vector = np.array([0.2, 0.4, 0.6, 0.3, 0.1, 0.05, 0.9, 0.8, 0.65, 0.4])
apple_vector = np.array([0.5, -0.3, 0.1, 0.0, -0.7, -0.4, 0.2, 0.1, 0.3, 0.4])


Find the similar word

In [21]:
similar_words = GoogleModel.most_similar(word)
print(similar_words)

[('Viet_Nam', 0.7901363968849182), ('Vietnam_War', 0.7331193089485168), ('Indochina', 0.6708170771598816), ('Vietnamese', 0.6704345941543579), ('Vietman', 0.6269871592521667), ('Cambodia', 0.6265947818756104), ('VietNam', 0.6258699893951416), ('While_Megi_bypassed', 0.6208897233009338), ('Hanoi', 0.6203270554542542), ('Laos', 0.6100524663925171)]


#Convert to list token of vocabulary

- There will be 2 columns are Review and Liked.
- The aim of this sesison will automatically analyze the review of customer to detect if they like their meal or vice versar

In [19]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing import sequence

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Review'])
sequences = tokenizer.texts_to_sequences(df['Review'])

maxlen = 100
X = sequence.pad_sequences(sequences, maxlen=maxlen)
# print('X:\n', X[:10])
print(X.shape)

(1000, 100)


Summary:
Goal: This code tokenizes text data (reviews) into sequences of integers and ensures that all sequences have the same length by padding or truncating them to 100 words.

Key Steps:

Tokenization (mapping words to integers).

Converting text reviews to sequences of integers.

Padding or truncating the sequences to ensure uniform length for neural network input.

#Create ember maxtrix

In [20]:
import numpy as np

embedding_dim = 300
word_index = tokenizer.word_index
num_words = min(len(word_index) + 1, len(GoogleModel.index_to_key))
embedding_matrix = np.zeros((num_words, embedding_dim))

print('num_words:', num_words)
for word, i in word_index.items():
    if i >= num_words:
        continue
    if word in GoogleModel.index_to_key:
        embedding_matrix[i] = GoogleModel.word_vec(word)


num_words: 2072


<ipython-input-20-d7eb5b195804>:13: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  embedding_matrix[i] = GoogleModel.word_vec(word)


Objective: The code builds an embedding matrix where each row corresponds to a word in your tokenizer’s vocabulary, and the row contains the 300-dimensional vector from the pre-trained Google Word2Vec model. If a word is not found in the Word2Vec model, that row remains filled with zeros (which could be handled later).

Purpose: This embedding matrix is typically used to initialize the embedding layer of a neural network, allowing the network to start with pre-trained word vectors rather than learning them from scratch, which often improves performance, especially when dealing with small datasets.

In [21]:
import numpy as np
import pandas as pd


embedding_dim = 300
word_index = tokenizer.word_index
num_words = min(len(word_index) + 1, len(GoogleModel.index_to_key))
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i >= num_words:
        continue
    if word in GoogleModel.index_to_key:
        embedding_matrix[i] = GoogleModel.word_vec(word)

# Save the final embedding matrix
np.save('embedding_matrix.npy', embedding_matrix)

# Optionally, save as a CSV for easier inspection
embedding_df = pd.DataFrame(embedding_matrix)
embedding_df.to_csv('embedding_matrix.csv', index=False)

print("Embedding matrix has been saved to 'embedding_matrix.npy' and 'embedding_matrix.csv'")


<ipython-input-21-4023458373c4>:14: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  embedding_matrix[i] = GoogleModel.word_vec(word)


Embedding matrix has been saved to 'embedding_matrix.npy' and 'embedding_matrix.csv'


#Create model

Embedding Layer: Transforms word indices into dense vectors.

Bidirectional LSTM: Captures sequence information in both forward and backward directions.

Flatten: Converts the sequence of LSTM outputs into a single vector.

Dense Layer: Outputs a probability for binary classification.

Compilation: Sets up the optimization and loss calculation for training.

In [22]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional

model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(maxlen, ))))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │         621,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 621,600 (2.37 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 621,600 (2.37 MB)

#Create training and testing data

In [23]:
print(df['Liked'])

0      1
1      0
2      0
3      1
4      1
      ..
995    0
996    0
997    0
998    0
999    0
Name: Liked, Length: 1000, dtype: int64


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df['Liked'], test_size=0.3, random_state=101)



#Start Training

In [26]:
model.fit(X_train, y_train, epochs=10, batch_size=32)
model.save('sentiment_model.h5')

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 165ms/step - acc: 0.5779 - loss: 0.6827
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 166ms/step - acc: 0.7802 - loss: 0.4841
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 270ms/step - acc: 0.8370 - loss: 0.3695
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 169ms/step - acc: 0.8751 - loss: 0.2833
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 176ms/step - acc: 0.9314 - loss: 0.1883
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 201ms/step - acc: 0.9410 - loss: 0.1588
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - acc: 0.9374 - loss: 0.1660
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 273ms/step - acc: 0.9605 - loss: 0.1162
Epoch 9/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 167ms/step - acc: 0.9762 - loss: 0.0851
Epoch 10/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 278ms/step - acc: 0.9774 - loss: 0.0658


#Evaluation

In [18]:
predictions = model.predict(X_test)
print('predict:', predictions[:3])
predictions = np.round(predictions)
print('predictions:', predictions.flatten())
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step
predict: [[0.92018294]
 [0.99666727]
 [0.5660553 ]]
predictions: [1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1.
 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0.
 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1.
 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0.
 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1.
 1. 1. 0. 0. 1. 1.

Validate the data

In [32]:
from keras.models import load_model

# Load the model from the file
loaded_model = load_model('sentiment_model.h5')

import numpy as np
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

# Assuming 'tokenizer' was used for your training data
# Also assume you know the 'maxlen' and 'embedding_dim' used before

# New review for evaluation
new_review = ["great"]

# Convert the new review to a sequence of integers
new_sequence = tokenizer.texts_to_sequences(new_review)

# Pad the sequence to match the input shape of the model
new_padded_sequence = pad_sequences(new_sequence, maxlen=maxlen)

# Use the loaded model to make a prediction
prediction = loaded_model.predict(new_padded_sequence)

# Output the result
print("Prediction (1 = Liked, 0 = Not Liked):", round(prediction[0][0]))



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
Prediction (1 = Liked, 0 = Not Liked): 1
